In [2]:
import pandas as pd
import numpy as np
import json
import os

In [38]:
# Need to modify absolute path
missing = pd.read_csv("D:\Courses\Pattern Recognition\WLASL\WLASL\data_download\scripts\missing.txt")

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Yaoga\AppData\Local\Temp\ipykernel_7820\1809448572.py:2: SyntaxWarning: invalid escape sequence '\C'
  missing = pd.read_csv("D:\Courses\Pattern Recognition\WLASL\WLASL\data_download\scripts\missing.txt")


In [40]:
# Need to modify absolute path
content = json.load(open(r'D:\Courses\Pattern Recognition\WLASL\WLASL\data_download\scripts\WLASL_v0.3.json'))
available_filenames = set(os.listdir('./dataset/videos'))

In [41]:
matched_samples = []
for entry in content:
    instances = entry['instances']
    gloss = entry['gloss']

    for inst in instances:
        video_id = inst['video_id']
        if video_id + '.mp4' in available_filenames:
            matched_samples.append([video_id, gloss])

with open('matched_samples.txt', 'w') as f:
    f.write("video_file\tgloss\n")
    for video_file, gloss in matched_samples:
        f.write(f"{video_id}\t{gloss}\n")

In [43]:
samples_df = pd.DataFrame(matched_samples)
samples_df

,0,1
0,69241,book
1,07069,book
2,07068,book
3,07070,book
4,07074,book
...,...,...
9759,63044,wheelchair
9760,63046,wheelchair
9761,63047,wheelchair
9762,63186,whistle


In [44]:
dataset = []
base_path = "./dataset/matrix"
for video, gloss in matched_samples:
    file_path = os.path.join(base_path, f"{video}.npy")
    if os.path.exists(file_path):
        matrix = np.load(os.path.join(base_path, f"{video}.npy"))
        dataset.append([gloss, matrix])
       
np.save("./dataset/dataset_all.npy", np.array(dataset, dtype=object))

In [45]:
input_all = np.load("./dataset/dataset_all.npy", allow_pickle=True)
input_all

array([['book',
        array([[ 0.5002426 ,  0.2930345 , -0.65388024, ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.5002483 ,  0.29436678, -0.79956585, ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.50024545,  0.29461762, -0.757012  , ...,  0.        ,
                 0.        ,  0.        ],
               ...,
               [ 0.4983627 ,  0.30170536, -0.7504038 , ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.49837023,  0.301757  , -0.7592436 , ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.49837747,  0.30184475, -0.7819647 , ...,  0.        ,
                 0.        ,  0.        ]], dtype=float32)              ],
       ['book',
        array([[ 0.539147  ,  0.26695675, -0.70452255, ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.53871536,  0.26878867, -0.67531997, ...,  0.        ,
                 0.        ,  0

In [ ]:
dataset_padding = []
# First pass to find the maximum number of frames (T)
T_max = 0
base_path = "./dataset/matrix"
for video, _ in matched_samples:
    file_path = os.path.join(base_path, f"{video}.npy")
    if os.path.exists(file_path):
        matrix = np.load(file_path)
        T_max = max(T_max, matrix.shape[0])  # update max frame length

print(f"Maximum number of frames: {T_max}")

# Second pass: pad matrices to T_max
for video, gloss in matched_samples:
    file_path = os.path.join(base_path, f"{video}.npy")
    if os.path.exists(file_path):
        matrix = np.load(file_path)
        T, D = matrix.shape  # T: frames, D: features (228)
        
        # Pad with zeros if needed
        if T < T_max:
            pad_amount = T_max - T
            pad_matrix = np.zeros((pad_amount, D), dtype=np.float32)
            matrix = np.vstack([matrix, pad_matrix])  # [T_max, D]

        dataset_padding.append([gloss, matrix])

# Save as object array
np.save("./dataset/dataset_all_padding.npy", np.array(dataset_padding, dtype=object))

Maximum number of frames: 233


In [48]:
input_padding = np.load("./dataset/dataset_all_padding.npy", allow_pickle=True)
input_padding

array([['book',
        array([[ 0.5002426 ,  0.2930345 , -0.65388024, ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.5002483 ,  0.29436678, -0.79956585, ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.50024545,  0.29461762, -0.757012  , ...,  0.        ,
                 0.        ,  0.        ],
               ...,
               [ 0.        ,  0.        ,  0.        , ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        , ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        , ...,  0.        ,
                 0.        ,  0.        ]], dtype=float32)              ],
       ['book',
        array([[ 0.539147  ,  0.26695675, -0.70452255, ...,  0.        ,
                 0.        ,  0.        ],
               [ 0.53871536,  0.26878867, -0.67531997, ...,  0.        ,
                 0.        ,  0

In [ ]:
data_padding = np.load(r'D:\Courses\Pattern Recognition\UB-CSE-555\dataset\dataset_all_padding.npy', allow_pickle=True)
data = np.load(r'D:\Courses\Pattern Recognition\UB-CSE-555\dataset\dataset_all.npy', allow_pickle=True)

np.savez_compressed('dataset_all_padding.npz', data=data_padding)
np.savez_compressed('dataset_all.npz', data=data)